# OTOT-C: ICPC WF Statistics

* Datasets:
  * `icpc-full.csv`: https://www.kaggle.com/datasets/justinianus/icpc-world-finals-ranking-since-1999
  * `region-mapping.csv`: custom data to map countries to regions

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

plt.ioff()

In [38]:
df = pd.read_csv('icpc-full.csv').reset_index()
region_mapping = pd.read_csv('region-map.csv')
df = pd.merge(df, region_mapping, on='Country')
df[df['Region'].isna()]

,index,Year,Date,Host,City,Venue,Rank,University,Country,Team,...,Gold,Silver,Bronze,Honorable,Score,Total,Score Percentage,Penalty,Prize,Region


In [39]:
# Number of teams per country that received a medal (Ranks 1-12).

fig, ax = plt.subplots(figsize=(10,10))
fig.suptitle('Number of Medalists per Country')

medal_team_per_country = df[df['Rank'] <= 12]['Country'].value_counts()
medal_team_per_country = pd.concat([
    medal_team_per_country.head(11),
    pd.Series({'Others': medal_team_per_country.tail(medal_team_per_country.size - 11).sum()})
])

patches, texts, autotexts = ax.pie(
    medal_team_per_country,
    labels=medal_team_per_country.index,
    rotatelabels=True,
    autopct='%1.2f%%',
    colors=plt.cm.Pastel2.colors,
    explode=([0.05] * (medal_team_per_country.size - 1)) + [0.1]
)

patches[-1].set_color('whitesmoke')
patches[-1].set_edgecolor('grey')
patches[-1].set_linewidth(0.1)

text_sizes = medal_team_per_country.values / medal_team_per_country.values[0] * 8 + 8
for i in range(len(texts)):
    t = texts[i]
    t.set_color('grey')
    t.set_size(text_sizes[i])
for t in autotexts:
    t.set_color('grey')
    t.set_size('7')

# plt.show()
fig.savefig('medal.png')
plt.close()


## Number of Medalists per Country

We want to see which countries has the most successful teams that receives a medal. In this case, we can just check for rank <= 12.

![image](medal.png)

Here we observe that more than a quarter of medalists are teams from Russia. The next two top countries are from China and United States.



In [40]:
# Number of problem solved

# Average number of problems solved
aver = df.loc[:, ['Year', 'Score']]
aver = aver.rename(columns={'Score': 'Average'})
aver = aver.groupby('Year').mean()

# Average number of problems solved by top 50
aver_top50 = df[df['Rank'] <= 50].loc[:, ['Year', 'Score']]
aver_top50 = aver_top50.rename(columns={'Score': 'Average (Top 50)'})
aver_top50 = aver_top50.groupby('Year').mean()

# Average number of problems solved by top 12
aver_top12 = df[df['Rank'] <= 12].loc[:, ['Year', 'Score']]
aver_top12 = aver_top12.rename(columns={'Score': 'Average (Top 12)'})
aver_top12 = aver_top12.groupby('Year').mean()

# Max number of problems to be solved
max_problems = df.loc[:, ['Year', 'Total']]
max_problems = max_problems.rename(columns={'Total': 'Max Problems'})
max_problems = max_problems.groupby('Year').first()

aver_comb = aver.merge(aver_top50, left_on='Year', right_on='Year')
aver_comb = aver_comb.merge(aver_top12, left_on='Year', right_on='Year')
aver_comb = aver_comb.merge(max_problems, left_on='Year', right_on='Year')

In [41]:
# Number of problems solved (Percentage)

# Average number of problems solved
aver_pcnt = df.loc[:, ['Year', 'Score Percentage']]
aver_pcnt = aver_pcnt.rename(columns={'Score Percentage': 'Average'})
aver_pcnt = aver_pcnt.groupby('Year').mean()

# Average number of problems solved by top 50
aver_pcnt_top50 = df[df['Rank'] <= 50].loc[:, ['Year', 'Score Percentage']]
aver_pcnt_top50 = aver_pcnt_top50.rename(columns={'Score Percentage': 'Average (Top 50)'})
aver_pcnt_top50 = aver_pcnt_top50.groupby('Year').mean()

# Average number of problems solved by top 12
aver_pcnt_top12 = df[df['Rank'] <= 12].loc[:, ['Year', 'Score Percentage']]
aver_pcnt_top12 = aver_pcnt_top12.rename(columns={'Score Percentage': 'Average (Top 12)'})
aver_pcnt_top12 = aver_pcnt_top12.groupby('Year').mean()

aver_pcnt_comb = aver_pcnt.merge(aver_pcnt_top50, left_on='Year', right_on='Year')
aver_pcnt_comb = aver_pcnt_comb.merge(aver_pcnt_top12, left_on='Year', right_on='Year')

In [42]:
fig, (ax0, ax1) = plt.subplots(2, 1, figsize=(20,10), sharex=True)
fig.suptitle('Average Problems Solved')

aver_comb.plot(kind='bar', ax=ax0, color=plt.cm.Pastel2.colors)
aver_pcnt_comb.plot(kind='bar', ax=ax1, color=plt.cm.Pastel2.colors, rot=30)

ax1.set_ylim([0.0, 1.0])
ax1.grid(axis='y')
ax1.set_axisbelow(True)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))

# plt.show()
fig.savefig('problem.png')
plt.close()

## Average Problems Solved

Here we want to see the different performances between those in the top 12, top 50 and everyone.

![image](problem.png)

Here we can see that the top 50 solves about 50% of the problems, with much variance depending on the year.

Years 2006 and 2014 are especially brutal, where even the average number of problems solved by the top 12 is under or at least close to 50%.

In [43]:
fig, (ax0, ax1) = plt.subplots(2, 1, figsize=(10,20))
fig.suptitle("Region Performances")
ax0.title.set_text("Average Problems Solved per Region")
ax1.title.set_text("Highest Ranks in the Region")

aver_region_pcnt = df.loc[:, ['Region', 'Score Percentage']]
aver_region_pcnt = aver_region_pcnt.rename(columns={'Score Percentage': 'Average'})
aver_region_pcnt = aver_region_pcnt.groupby('Region').mean().sort_values(by='Average', ascending=False)

ax0.bar(
    aver_region_pcnt.index,
    aver_region_pcnt['Average'],
    color=plt.cm.Pastel2.colors
)

# aver_region_pcnt.T.plot(kind='bar', ax0=ax0, color=plt.cm.Pastel2.colors)

ax0.set_xticklabels(ax0.get_xticklabels(), rotation=30, ha='right')

ax0.set_ylim([0.0, 1.0])
ax0.grid(axis='y')
ax0.set_axisbelow(True)
ax0.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))

rank_percentile = df.groupby(['Year'])['Rank'].transform(lambda x: x / x.count()).rename("Rank Percentile").reset_index()
rank_percentile = pd.merge(df, rank_percentile)

g = rank_percentile.groupby(['Region'])
rank_percentile = pd.DataFrame({col: val['Rank Percentile'] for col, val in g})

rank_percentile = rank_percentile[aver_region_pcnt.index]
rank_percentile.plot(kind='box',ax=ax1)

labels = [label.get_text().replace('(','').replace(',)','') for label in ax1.get_xticklabels()]
ax1.set_xticklabels(labels, rotation=30, ha='right')

ax1.invert_yaxis()
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))

# plt.show()
fig.savefig('region.png')
plt.close()

/tmp/ipykernel_9986/1566467493.py:18: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax0.set_xticklabels(ax0.get_xticklabels(), rotation=30, ha='right')


## Region Performances

Here we want to see how each region compares with each other.

![image](region.png)

One interesting thing to note is that North America, consisting of USA and Canada, despite having a lot of medalists has a lower performance in general.

Despite not having any dominating countries, Europe and Asia Pacific have relatively high performances. We can infer that these regions generally have a lot of countries that are high-performing over the years.